## Operational Space Control (Khatib)

1. The controller is defined as $\Gamma = \sum_{k=1}^N J_{k|prec(k)}^T F_{k|prec(k)}$
2. The prioritized jacobian for task $k$ is given by $J_{k|prec(k)} := J_k N_{prec(k)}$
3. $N_{prec(k)} = I - \sum_{i=1}^{k-1} \bar J_{i|prec(i)} J_{i|prec(i)}$
4. $N_{prec(k)} = N_{k-1|prec(k-1)} N_{k-2|prec(k-2)} ... I$. See *sentis A.24*. This is equivalent to $N_{prec(k)} = N_{k-1|prec(k-1)} N_{prec(k-1)}$

##### Algorithm
1. Initialize $N_{prec(1)} = I$, $priority = 1$
2. $N_{1|prec(1)} \leftarrow (I-\bar J_1 J_1)$
3. For $t_i \in T$
<br> a) If $priority(t_i) > priority$,
<br> $$N_{prec(i)} = N_{i-1|prec(i-1)} N_{prec(i-1)}$$
<br> $$priority = priority(t_i)$$ 
<br> b) Set $N_{prec(i)}$ in current task
<br> c) Calculate $J_{i|prec(i)} := J_i N_{prec(i)}$
<br> d) Calculate $N_{i|prec(i)} := (I-\bar J_{i|prec(i)} J_{i|prec(i)})$

##### Example (3 task control)
$$\Gamma = J_{1|prec(1)}^T F_{1|prec(1)} + J_{2|prec(2)}^T F_{2|prec(2)} + J_{3|prec(3)}^T F_{3|prec(3)}$$
$$\Gamma = J_{1}^T F_{1} + (J_{2}N_{prec(2)})^T F_{2|prec(2)} + (J_{3}N_{prec(3)})^T F_{3|prec(3)}$$
$$\Gamma = J_{1}^T F_{1} + (J_{2}(I-\bar J_{1} J_{1}))^T F_{2|prec(2)} + (J_{3} (I-\bar J_{2|1} J_{2|1}) (I-\bar J_{1} J_{1}))^T F_{3|prec(3)}$$

#### Dynamically Consistent Jacobian Inverse

1. The heart of operational space control is the implicit constrained minimization of the kinetic energy that is embedded within the dynamically consistent jacobian inverse $\bar J$. We can appreciate this more clearly with the use of lagrange multipliers applied to the following problem: $$\min_{\dot q} \frac{1}{2} \dot q^T M \dot q$$ $$st. \dot x = J \dot q$$
2. The lagrangian is given by $L = \dot q^T M \dot q - \lambda^T (\dot x - J \dot q)$ giving the partial derivatives $\frac{\partial L}{\partial \dot q} = M \dot q - J^T \lambda = 0$ and the trivial constraint equation $\frac{\partial L}{\partial \dot \lambda} = \dot x - J \dot q = 0$. Premultiplying the first equation by $J$ and substituting the second, we have $J \dot q = (J M^-1 J^T) \lambda = \Lambda^{-1} \lambda \rightarrow \lambda = \Lambda \dot x$. Putting this result back to the first equation, we have $\dot q = M^{-1} J^T \Lambda \dot x = \bar J \dot x \rightarrow \bar J = M^{-1} J^T \Lambda$.

#### Properties of nullspace projectors
Any nullspace projector $N = I-J^\#J$ satisfies:
<br> *Idempotence* : $NNNNN ... = N$, **Proof** := check $(I-J^\#J)(I-J^\#J) = I-J^\#J$ using property $J J^\# J=J$
<br> *(Dynamically consistent projectors) Commutation with $M^{-1}$* : $N M^{-1} = M^{-1} N^T$ **Proof** := check $(I-\bar J J) M^{-1} = M^{-1}(I-\bar J J)^T = M^{-1} J^T \Lambda J M^{-1}$
<br> *(Dynamically consistent projectors) Commutative* : $N_{i|...1} N_{j|...1} = N_{j|...1} N_{i|...1}, \forall i,j \in \mathbb{R}^+$, **Proof** := $(I-\bar J_{j...|2|1} J_{j...|2|1}) (I-\bar J_{i...|2|1} J_{i...|2|1}) = I - \bar J_{j...|2|1} J_{j...|2|1} - \bar J_{i...|2|1} J_{i...|2|1} + J_{j...|2|1} J_{j...|2|1} \bar J_{i...|2|1} J_{i...|2|1}$. Simplified proof, for two tasks, the last term gives $\bar J_1 J_1\bar J_{2|1} J_{2|1} = \bar J_1 J_1 M^{-1} N_1^T J_2^T  J_{2|1} = 0$ using the property above. Similarly, $\bar J_{2|1} J_{2|1} \bar J_1 J_1 = \bar J_{2|1} J_2 N_1 M^{-1} J_1^T \Lambda_1 J_1 = 0$

#### Task dynamics in Stack of Tasks
Consider the proposed closed loop control (in free space) using:
$$M \ddot q + c + g = J_1^T F_1 + J_{2|1}^T F_{2|1} + J_{3|2|1}^T F_{3|2|1}$$

Premultiplying with $J_2 M^{-1}$ exposes the task dynamics for the second task:
$$\ddot x - \dot J \dot q + J_2 (c + g) = J_2 M^{-1} J_1^T F_1 + J_2 M^{-1} J_{2|1}^T F_{2|1} + J_2 M^{-1} N_2^T J_{3}^T F_{3|2|1}$$
The last term goes to $0$ using $J_2 M^{-1} N_2^T = J_2 N_2 M^{-1} = 0$
$$\ddot x_2 - \dot J_2 \dot q + J_2 M^{-1} (c + g) = J_2 M^{-1} J_1^T F_1 + J_2 N_1 M^{-1} N_1^T J_{2}^T F_{2|1}$$
$$\ddot x_2 - \dot J_2 \dot q + J_2 M^{-1} (c + g) = J_2 M^{-1} J_1^T F_1 + \Lambda_{2|1}^{-1} F_{2|1}$$

Now to control the double integrator system of $\ddot x_2$, we use the feedback linearizing control law:
$F_{2|1} = \Lambda_{2|1} \alpha + \Lambda_{2|1}(J_2 M^{-1}c-\dot J_2 \dot q) + \Lambda_{2|1}J_2 M^{-1} g -\Lambda_{2|1} J_2 M^{-1} J_1^T F_1$
<br> - The first term results in $\ddot x_2 = \alpha$ when the control law is applied to the $J_2$ submanifold constrained dynamics. 
<br> - The second term is the $\mu_{2|1}$ term
<br> - The third term is the $p_{2|1}$ term
<br> - The last term is the decoupling term that removes the accelerations from higher priority tasks.

### Task Switching (Niels Dehio)
1. The task switching method can be implemented via a projection using the range and null spaces.
2. Properties of projection matrices:
<br> A square projection matrix $X$ is idempotent by definition, i.e. $X = X^2$
<br> It is an orthogonal projection iff $X = X^T$ otherwise the projection is referred to as oblique
<br> $X$ and $Y$ project onto orthogonal subspaces iff $X^TY = 0$
<br> Given orthonormal basis $B$, the range and nullspaces of $B$ defined by $C=BB^T$ and $N=I-BB^T$

## Optimization-based robot control (Andrea Del Prete)
1. Inverse dynamics control can be expressed as a constrained optimization problem to include limits:
$$\min_{\tau,\ddot q} ||\ddot q - \ddot q_d||^2$$
$$s.t. M \ddot q + h = \tau$$
$$\tau_{min} \leq \tau \leq \tau_{max}$$
$$i_{min} \leq k_{\tau} \tau \leq i_{max}$$

2. Since torques and accelerations are optimization variables in least squares program, we cannot include velocity and position constraints. To do so, we can assume constant acceleration and have:
$$v_{min} \leq v + \Delta t \dot v \leq v_{max}$$
$$q_{min} \leq q + \Delta t \dot v + \frac{1}{2} \dot v \Delta t^2 \leq q_{max}$$

3. The position limits can lead to high accelerations as especially in the limits and is typically incompatible with torque/current limits.

4. For task space, error dynamics need to be expressed as affine functions, for example, for end-effector positions, we have:
$$\ddot e = -K_p e - K_d \dot e$$
$$\ddot x - \ddot x_d = -K_p e - K_d \dot e$$
$$J \ddot q + \dot J \dot q - \ddot x_d = -K_p e - K_d \dot e$$
$$J \ddot q =- \dot J \dot q + \ddot x_d -K_p e - K_d \dot e \Longrightarrow A_q \ddot q = a$$

5. Generalizing, we have an additional linear constraint:
$$g(y) := \begin{bmatrix} A_q & A_\tau \end{bmatrix} \begin{bmatrix} \ddot q \\ \tau \end{bmatrix} - a \Longrightarrow Ay - a$$

6. In task space, our optimization problem leads to:
$$\min_{y = (\ddot q, \tau)} ||Ay - a||^2$$
$$s.t. \begin{bmatrix} M & -S^T \end{bmatrix} y = -h$$

7. Integrating a soft contact, use an estimated $\hat f$:
$$\min_{y = (\ddot q, \tau)} ||Ay - a||^2$$
$$s.t. \begin{bmatrix} M & -S^T \end{bmatrix} y = -h + J^T \hat f$$
This only works if the contact is actually soft, otherwise the mismatch can lead to numerical stability issues.

8. Integrating rigid contact as a constraint can be done by specifying the constraint manifold $c(q) = constant$, i.e. that contacts do not move. Thus, $J_c \dot q = 0$ and $J_c \ddot q + \dot J_c \dot q = 0$. Incorporating this into our optimization problem:
$$\min_{y = (\ddot q, f, \tau)} ||Ay - a||^2$$
$$s.t. \begin{bmatrix} J_c & 0 & 0 \\ M & -J^T & -S^T \end{bmatrix} y = \begin{bmatrix} -J_c \ddot q \\ -h \end{bmatrix}$$

9. For multi-task control, a hierarchical LSP can be used where each lower priority task (with priority $i$) will have an additional constraint $g_j (y) = g_j^*, \forall j < i$

10. Computational cost is always dominated by the Hessian (Cholesky decomposition) which taks $O(n^3)$. To reduce the computation time, we can exploit the structure of the problem, in particular, we can decompose the active and passive joint torques into:
$$\begin{bmatrix} J_c & 0 & 0 \\ M_u & -J_u^T & -0 \\ M_a & -J_a^T & -I  \end{bmatrix} \begin{bmatrix} \ddot q \\ f \\ \tau \end{bmatrix}  = \begin{bmatrix} -J_c \ddot q \\ -h_u \\ -h_a \end{bmatrix}$$
We can re-express $y$ as:
$$\begin{bmatrix} \ddot q \\ f \\ \tau \end{bmatrix} = \begin{bmatrix} I & 0 \\ 0 & I \\ M_a & -J_a^T \end{bmatrix} \begin{bmatrix} \ddot q \\ f \end{bmatrix} + \begin{bmatrix} 0 \\ 0 \\ -h_a \end{bmatrix}\Longrightarrow y = D \bar y + d$$

11. The original problem is:
$$\min_{y = (\ddot q, f, \tau)} ||Ay - a||^2$$
$$s.t. By \leq b$$
$$\begin{bmatrix} J_c & 0 & 0 \\ M & -J^T & -S^T \end{bmatrix} y = \begin{bmatrix} -J_c \ddot q \\ -h \end{bmatrix}$$
The reformulated problem is:
$$\min_{\bar y} ||A D \bar y + A d - a||^2$$
$$s.t. B D \bar y \leq b - Bd$$
$$\begin{bmatrix} J_c & 0 \\ M_u & - J_u^T \end{bmatrix} \begin{bmatrix} \ddot q \\ f \end{bmatrix} = \begin{bmatrix} -J_c \ddot q \\ -h_u \end{bmatrix}$$

## Hierarchical Quadratic Programming (Mansard)
1. Task space inverse dynamics can be solved via a LSP using the following:
$$\min_{y = (\ddot q, \tau, w)} ||w||^2$$
$$s.t. M \ddot q + h = \tau$$
$$\ddot e^* + w = J \ddot q + \dot J \dot q$$

2. Substituting the second constraint back into the objective function, we can eliminate $w$:
$$\min_{y = (\ddot q, \tau)} ||J \ddot q - (-\dot J \dot q+(\ddot x_d -K_p e - K_d \dot e ))||^2$$
$$s.t. M \ddot q + h = \tau$$
This is similar to what was proposed by *Del Prete*. Using the slack variable $w$ is useful to add some freedom in the HQP case.

3. Both approaches uses the minimum norm solution for $\ddot q$ to satisfy $J \ddot q + \dot J \dot q = \ddot x_{ref} = \ddot x_d -K_p e - K_d \dot e$

## Task Switching (Niels Dehio)
1. The task switching method can be implemented via a projection using the range and null spaces.
2. 